### Build a Simple LLM Application with LCEL (This is not using RAG)
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

Here we will be accessing the LLM models in Groq Cloud. You need to access the API keys

Models available in Groq Cloud - https://console.groq.com/docs/models

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
### Importing the GROQ API KEY

groq_api_key = os.getenv('GROQ_API_KEY')

To access the models available in Groq Cloud we need to have  

1) Create a Groq account
2) Get an API key
3) Install the langchain-groq integration package.

Reference Documentation - https://python.langchain.com/docs/integrations/chat/groq/

In [7]:
from langchain_groq import ChatGroq 

In [8]:
model = ChatGroq(model='gemma2-9b-it' , groq_api_key = groq_api_key)

In [9]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024D81333C10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024DFEF38110>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

Once the model is initialized we have to pass the question to the model. We can do it in multiple ways.


Method 1 - By constructing messages and then passing it to the model

In langchain_core we will be using HumanMessage,SystemMessage

- SystemMessage - This will instruct the LLM to behave in specific way

- HumanMessage  - This will provide the question from human to the LLM model



In [10]:
from langchain_core.messages import HumanMessage,SystemMessage

In [11]:
messages = [SystemMessage(content='Translate the following from English to French'),
HumanMessage(content='Hello How are you?')]

In [17]:
print(model.invoke(messages))

content='Here are a couple of ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal) \n\n\nLet me know if you\'d like more translations!\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 21, 'total_tokens': 83, 'completion_time': 0.112727273, 'prompt_time': 0.000335649, 'queue_time': 0.01422053, 'total_time': 0.113062922}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-79d5d7d2-c9f9-4492-b9a5-6d1f7059b58e-0' usage_metadata={'input_tokens': 21, 'output_tokens': 62, 'total_tokens': 83}


We can use String Output Parser to display the output from the LLM

Documentation - https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html

In [18]:
from langchain_core.output_parsers import StrOutputParser

In [19]:
parser = StrOutputParser()

In [20]:
parser.invoke(model.invoke(messages))

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends/family)\n\nThe best option depends on the context and your relationship with the person you\'re speaking to. \n\n\n'

We can also use LCEL to create chain. This chain will be a sequence of events. 

In [22]:
chain = model | parser

chain.invoke(messages)

'Here\'s the translation, along with some variations:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most formal way to say hello and ask how someone is doing.)\n\n**Informal:**\n\n* **Salut, ça va ?** (This is a very common and casual way to say hello and ask how someone is doing.)\n* **Coucou, comment vas-tu ?** (This is a more friendly and informal way to say hello and ask how someone is doing. "Coucou" is a French interjection similar to "hey" or "hi".)\n\nLet me know if you\'d like to know how to say "I\'m well, thank you" in French! \n\n\n'

Method 2 - Using Chat Prompt Template

In [23]:
from langchain_core.prompts import ChatPromptTemplate

In [26]:
prompt = ChatPromptTemplate.from_messages(
    [('system','Translate the following into {language}'),
    ('user' , '{text}')]
)

In [28]:
prompt.invoke({'language' : 'French' , 'text' : 'Hello'})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [29]:
### To see the prompt in list format as earlier

result = prompt.invoke({'language' : 'French' , 'text' : 'Hello'})
result.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

Now once the Prompt Template is designed we can pass it to the LLM using LCEL by creating chains

In [30]:
chain = prompt | model | parser

In [31]:
chain.invoke({'language' : 'French' , 'text' : 'Hello'})

"Bonjour \n\n\nLet me know if you'd like me to translate anything else!  \n"